Download dataset

In [162]:
!wget "https://coinmetrics.io/data/eth.csv" -O eth.csv

--2018-08-13 20:06:16--  https://coinmetrics.io/data/eth.csv
Resolving coinmetrics.io (coinmetrics.io)... 139.59.153.246
Connecting to coinmetrics.io (coinmetrics.io)|139.59.153.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187010 (183K) [text/csv]
Saving to: ‘eth.csv’

eth.csv             100%[===================>] 182.63K   139KB/s    in 1.3s    

2018-08-13 20:06:18 (139 KB/s) - ‘eth.csv’ saved [187010/187010]



Import dependencies

In [77]:
import csv
import dateutil

Load data into an array

In [78]:
eth_data = []

with open('eth.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        eth_data.append(row)

print(",".join(eth_data[0])) # columns
print("\n")

eth_data = eth_data[1:] # remove column names row
print(",".join(eth_data[0])) # print first row

date,txVolume(USD),adjustedTxVolume(USD),txCount,marketcap(USD),price(USD),exchangeVolume(USD),generatedCoins,fees,activeAddresses,medianTxValue(USD),medianFee,averageDifficulty,paymentCount,blockSize,blockCount


2015-07-30,,,,,,,39311.09375,,,,,1.21335493201e+11,,4449897,6911,


Create structured data array

In [118]:
class EthDataPoint:
    def __init__(self, date, price):
        self.date = dateutil.parser.parse(date)
        if price is "":
            self.price = None
        else:
            self.price = float(price)
        
    def __repr__(self):
        return f"EthDataPoint{{date={self.date},price={self.price}}}"

In [119]:
eth_dps_mapped = list(map(lambda row: EthDataPoint(row[0], row[5]), eth_data))
eth_dps = list(filter(lambda dp: dp.price != None, eth_dps_mapped))

Store transactions

In [133]:
class Tx:
    def __init__(self, buyOrSell, amount, date):
        self.typ = buyOrSell
        self.amount = amount
        self.date = date
    
    def __repr__(self):
        return f"Tx{{typ={self.typ},amount={self.amount},date={self.date}}}"

Traverse data and generate transactions

In [154]:
txs = []
lookback_days = 7

for i in range(720 + lookback_days, len(eth_dps)):
    start_price = eth_dps[i - lookback_days].price
    end_price = eth_dps[i].price
    diff_pct = round((end_price-start_price)/start_price * 10000) / 100
    print(eth_dps[i].date, diff_pct)
    
    if diff_pct < -20 and diff_pct >= -25:
        txs.append(Tx("sell", eth_dps[i].price, eth_dps[i].date))
    
    if diff_pct < -25:
        txs.append(Tx("buy", -eth_dps[i].price, eth_dps[i].date))
        
    if diff_pct > 20 and diff_pct <= 25:
        txs.append(Tx("buy", eth_dps[i].price, eth_dps[i].date))
    
    if diff_pct > 25:
        txs.append(Tx("sell", -eth_dps[i].price, eth_dps[i].date))

2017-08-03 00:00:00 7.48
2017-08-04 00:00:00 10.27
2017-08-05 00:00:00 15.26
2017-08-06 00:00:00 24.03
2017-08-07 00:00:00 32.34
2017-08-08 00:00:00 31.47
2017-08-09 00:00:00 31.11
2017-08-10 00:00:00 34.87
2017-08-11 00:00:00 30.7
2017-08-12 00:00:00 38.53
2017-08-13 00:00:00 21.04
2017-08-14 00:00:00 14.08
2017-08-15 00:00:00 11.46
2017-08-16 00:00:00 -2.62
2017-08-17 00:00:00 1.97
2017-08-18 00:00:00 2.45
2017-08-19 00:00:00 -4.06
2017-08-20 00:00:00 -4.42
2017-08-21 00:00:00 0.82
2017-08-22 00:00:00 7.03
2017-08-23 00:00:00 8.78
2017-08-24 00:00:00 4.84
2017-08-25 00:00:00 8.09
2017-08-26 00:00:00 12.09
2017-08-27 00:00:00 12.72
2017-08-28 00:00:00 16.49
2017-08-29 00:00:00 8.38
2017-08-30 00:00:00 17.19
2017-08-31 00:00:00 19.21
2017-09-01 00:00:00 17.59
2017-09-02 00:00:00 16.82
2017-09-03 00:00:00 4.75
2017-09-04 00:00:00 -0.83
2017-09-05 00:00:00 -14.48
2017-09-06 00:00:00 -15.07
2017-09-07 00:00:00 -11.87
2017-09-08 00:00:00 -14.04
2017-09-09 00:00:00 -23.63
2017-09-10 00:00:0

In [155]:
txs

[Tx{typ=buy,amount=256.42,date=2017-08-06 00:00:00},
 Tx{typ=sell,amount=-261.24,date=2017-08-07 00:00:00},
 Tx{typ=sell,amount=-269.1,date=2017-08-08 00:00:00},
 Tx{typ=sell,amount=-297.63,date=2017-08-09 00:00:00},
 Tx{typ=sell,amount=-296.96,date=2017-08-10 00:00:00},
 Tx{typ=sell,amount=-294.49,date=2017-08-11 00:00:00},
 Tx{typ=sell,amount=-308.71,date=2017-08-12 00:00:00},
 Tx{typ=buy,amount=310.37,date=2017-08-13 00:00:00},
 Tx{typ=sell,amount=296.17,date=2017-09-09 00:00:00},
 Tx{typ=buy,amount=-215.22,date=2017-09-15 00:00:00},
 Tx{typ=buy,amount=412.5,date=2017-11-24 00:00:00},
 Tx{typ=sell,amount=-475.68,date=2017-11-25 00:00:00},
 Tx{typ=sell,amount=-465.97,date=2017-11-26 00:00:00},
 Tx{typ=sell,amount=-471.53,date=2017-11-27 00:00:00},
 Tx{typ=sell,amount=-480.52,date=2017-11-28 00:00:00},
 Tx{typ=sell,amount=-473.28,date=2017-11-29 00:00:00},
 Tx{typ=sell,amount=-644.91,date=2017-12-13 00:00:00},
 Tx{typ=sell,amount=-700.59,date=2017-12-14 00:00:00},
 Tx{typ=sell,amount=

In [156]:
sum(map(lambda tx: tx.amount,txs))

-18451.040000000005